In [40]:
from dotenv import dotenv_values
import requests
import json
import random

config = dotenv_values("../.env")  # config = {"USER": "foo", "EMAIL": "foo@example.org"}
debug_server = config['DEBUG_SERVER']
token = config['TOKEN']
submitt_url = config['DEBUG_SERVER_SUBMIT']
HEADER = {'content-type': 'application/json'}

## DL Model

In [28]:
from data import DataModule
from models import Transformer
from transformers import RobertaTokenizerFast
import torch
import numpy as np

model_path = r"../models/transformer/model_val_loss0.28.ckpt"
tokenizer_path =  r"../models/roberta-tokenizer"

In [29]:
system1 = Transformer(
    ntokens=30000,
    emsize=128,
    d_hid=128,
    nlayers=1,
    nhead=2,
    dropout=0.2,
).load_from_checkpoint(model_path)
system1.eval()
system1_tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_path)


file ../models/roberta-tokenizer/config.json not found
file ../models/roberta-tokenizer/config.json not found


In [33]:
def get_prediction(text):
    text = system1_tokenizer.encode_plus(
    text,
    # add_special_tokens=True,
    max_length=512,
    padding="max_length",
    return_tensors="pt",
    return_attention_mask=False,
    truncation=True,
    )
    output = torch.sigmoid(system1(text['input_ids'])).detach().numpy()
    return np.heaviside(output, 0)

# Official Submission

## Workflow
```
getWritings
while request not empty:
    for run in runs:
        POST request
    getWritings
```

In [23]:
GET_URL = 'http://127.0.0.1:5000'
systems = [get_prediction, None, None, None, None]
ans = requests.get(GET_URL)
ans_dict = json.loads(ans.text)
while ans_dict != []:
    for run, system in enumerate(systems):
        post_url = config['T1_SUBMISSION'] + token + "/" + str(run)
        submission = []
        for ans in ans_dict:
            nick = ans['nick']
            text = ans['content']
            submission.append({
                'nick': nick,
                'decision': system(text),
                'score': round(random.uniform(0,4),1)
            })
        post_request = requests.post(post_url, data = json.dumps(submission), headers = HEADER)
    ans = requests.get(GET_URL)
    ans_dict = json.loads(ans.text)

[]
